# Valuation of bonds with credit spreads

In [1]:
import QuantLib as ql 

In [2]:
# set evaluation date

eval_date = ql.Date(26, ql.July, 2016)
ql.Settings.instance().evaluationDate = eval_date

In [3]:
# construct yield curve

flat_rate = ql.SimpleQuote(0.0015)
rate_handle = ql.QuoteHandle(flat_rate)
day_count = ql.Actual360()
calendar = ql.UnitedStates()

ts_yield = ql.FlatForward(eval_date, rate_handle, day_count)
ts_handle = ql.YieldTermStructureHandle(ts_yield)

In [4]:
# construct schedule

issue_date = ql.Date(15, ql.July, 2016)
maturity_date = ql.Date(15, ql.July, 2021)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates()
business_convention = ql.Unadjusted
date_generation = ql.DateGeneration.Backward
month_end = False

schedule = ql.Schedule(issue_date,
                       maturity_date,
                       tenor,
                       calendar,
                       business_convention,
                       business_convention,
                       date_generation,
                       month_end)

In [5]:
# construct bond

settlement_days = 0
day_count = ql.Thirty360()
coupon_rate = 0.03
coupons = [coupon_rate]
face_value = 100

fixed_rate_bond = ql.FixedRateBond(settlement_days,
                                   face_value,
                                   schedule,
                                   coupons,
                                   day_count)

In [6]:
# engine

bond_engine = ql.DiscountingBondEngine(ts_handle)
fixed_rate_bond.setPricingEngine(bond_engine)
fixed_rate_bond.NPV()

114.18461651948999

In [7]:
# assume that the market prices this bond with a 50BP spread on top of the treasury

flat_rate.setValue(0.0015 + 0.005)
fixed_rate_bond.NPV()

111.5097766266561

In [8]:
flat_rate.setValue(0.0015)
fixed_rate_bond.NPV()

114.18461651948999

## Parallel Shift of the Yield Curve

In [9]:
spread = ql.SimpleQuote(0.005)
spread_handle = ql.QuoteHandle(spread)
ts_spreaded = ql.ZeroSpreadedTermStructure(ts_handle, spread_handle)
ts_spreaded_handle = ql.YieldTermStructureHandle(ts_spreaded)

In [10]:
bond_engine = ql.DiscountingBondEngine(ts_spreaded_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

In [11]:
fixed_rate_bond.NPV()

111.50977662665609

In [12]:
# change in spread 

spread.setValue(0.01)
fixed_rate_bond.NPV()

108.89999943320038

## Non-Parallel Shift of the Yield Curve

In [13]:
spread1 = ql.SimpleQuote(0.005)
spread2 = ql.SimpleQuote(0.005)

start_date = eval_date
end_date = calendar.advance(start_date, ql.Period(50, ql.Years))

ts_spreaded = ql.SpreadedLinearZeroInterpolatedTermStructure(ts_handle,
                                                             [ql.QuoteHandle(spread1), ql.QuoteHandle(spread2)],
                                                             [start_date, end_date])

ts_spreaded_handle = ql.YieldTermStructureHandle(ts_spreaded)

In [14]:
bond_engine = ql.DiscountingBondEngine(ts_spreaded_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

In [15]:
fixed_rate_bond.NPV()

111.50977662665609

In [16]:
# parallel shift

spread1.setValue(0.01)
spread2.setValue(0.01)
fixed_rate_bond.NPV()

108.89999943320038

In [17]:
# non-parallel shift

spread1.setValue(0.005)
spread2.setValue(0.01)
fixed_rate_bond.NPV()

111.25358792334083